### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

purchase_data.head(10)


,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44
5,5,Yalae81,22,Male,81,Dreamkiss,3.61
6,6,Itheria73,36,Male,169,"Interrogator, Blood Blade of the Queen",2.18
7,7,Iskjaskst81,20,Male,162,Abyssal Shard,2.67
8,8,Undjask33,22,Male,21,Souleater,1.10
9,9,Chanosian48,35,Other / Non-Disclosed,136,Ghastly Adamantite Protector,3.58


## Player Count

* Display the total number of players


In [3]:
totalPlayers = purchase_data['SN'].nunique()
print('Total players:', totalPlayers)

Total players: 576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [4]:
purchase_data.nunique()

Purchase ID    780
SN             576
Age             39
Gender           3
Item ID        179
Item Name      179
Price          145
dtype: int64

In [5]:
avgAge = purchase_data['Age'].mean()
numItems = purchase_data['Item ID'].nunique()
purchaseCount = purchase_data['Purchase ID'].count()
revenueTotal = purchase_data['Price'].sum()


SummaryDF = pd.DataFrame()
SummaryDF['Average Age'] = [avgAge]
SummaryDF['Unique Number of Items']=[numItems]
SummaryDF['Total Number of Purchases'] = [purchaseCount]
SummaryDF['Total Revenue'] = [revenueTotal]

SummaryDF

,Average Age,Unique Number of Items,Total Number of Purchases,Total Revenue
0,22.714103,179,780,2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [6]:
#SN_DF drops all repeat instances of usernames, to get only 1 of each user
SN_DF = purchase_data.drop_duplicates(subset=['SN'])

genderDemo = pd.DataFrame(SN_DF['Gender'].value_counts())
genderDemo.rename(columns={'Gender':'Total Player Count'}, inplace=True)
genderDemo['Percent of Players']=(genderDemo['Total Player Count']/totalPlayers*100)
genderDemo


,Total Player Count,Percent of Players
Male,484,84.027778
Female,81,14.062500
Other / Non-Disclosed,11,1.909722


In [7]:
groupedGender = purchase_data.groupby('Gender')

genPurchaseCounts = pd.DataFrame(groupedGender['Purchase ID'].count())
genTotalRevenue = pd.DataFrame(groupedGender['Price'].sum())
purchaseAnalysisDF = pd.merge(genPurchaseCounts, genTotalRevenue, left_index=True, right_index=True)
purchaseAnalysisDF.rename(columns={'Purchase ID':'Purchase Count', 'Price':'Total Purchase Revenue'}, inplace=True)
purchaseAnalysisDF['Average Purchase Value']=purchaseAnalysisDF['Total Purchase Revenue']/purchaseAnalysisDF['Purchase Count']



In [21]:
#this section has no duplicate SNs
#groupedGender_bySN = SN_DF.groupby('Gender')
#purchaseAnalysisDF['Average Price per Person']= groupedGender_bySN.mean()['Price']
purchase_data_f = purchase_data[purchase_data['Gender']=='Female']
purchase_data_m = purchase_data[purchase_data['Gender']=='Male']
purchase_data_oth = purchase_data[purchase_data['Gender']=='Other / Non-Disclosed']

#find the average price per each person, divided by gender, put them in a list, and put it in the analysis DF
avg_per_f = pd.DataFrame(purchase_data_f.groupby('SN').mean())['Price'].mean()
avg_per_m = pd.DataFrame(purchase_data_m.groupby('SN').mean())['Price'].mean()
avg_per_oth = pd.DataFrame(purchase_data_oth.groupby('SN').mean())['Price'].mean()

avgs_per = [avg_per_f, avg_per_m, avg_per_oth]
purchaseAnalysisDF['Average Price per Person'] = avgs_per



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [22]:
purchaseAnalysisDF

,Purchase Count,Total Purchase Revenue,Average Purchase Value,Average Price per Person
Gender,,,,
Female,113,361.94,3.203009,3.194835
Male,652,1967.64,3.017853,3.014269
Other / Non-Disclosed,15,50.19,3.346000,3.348636


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [145]:
ageBins = [0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, 44.9, 49.9, 110]
ageBinLabels = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50+']

SN_DF['Age Group'] = pd.cut(SN_DF['Age'], ageBins, labels = ageBinLabels)

groupedbyAgeGroup = SN_DF.groupby('Age Group')

agePurchaseCount = pd.DataFrame(groupedbyAgeGroup.count()['Purchase ID'])
agePurchaseAvg = pd.DataFrame(groupedbyAgeGroup.mean()['Price'])
agePurchaseTotal = pd.DataFrame(groupedbyAgeGroup.sum()['Price'])
#agePurchaseAvg_SN = avg price per person


#merge to create 2 halves then merge the halves
ageDemo1 = pd.merge(agePurchaseCount, agePurchaseAvg, left_index=True, right_index=True)
ageDemo2 = pd.merge(agePurchaseTotal, agePurchaseAvg_SN, left_index=True, right_index=True)

ageDemoDF = pd.merge(ageDemo1, ageDemo2, left_index=True, right_index=True)
ageDemoDF

<ipython-input-145-dee17165217a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SN_DF['Age Group'] = pd.cut(SN_DF['Age'], ageBins, labels = ageBinLabels)


,Price
Age Group,
<10,57.63
10-14,67.64
15-19,331.88
20-24,790.39
25-29,223.93
30-34,151.92
35-39,108.81
40-44,34.75
45-49,1.70


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
